# Summary:
- No SMOTE
- Based on the 16x16 size vectors
- 1D Cov + two layers of LSTM is tested
- Validation set accuracy =  0.8576810380663019 same as that from the 32x32x3 vectors

In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [5]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [6]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 256), (14333,), (4778, 256), (4778,), 19111)

In [7]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [8]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],16,16) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],16,16) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [9]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [10]:
model = model(input_shape = (16,16))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 768)            185088    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 768)            3072      
_________________________________________________________________
activation_1 (Activation)    (None, 1, 768)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 768)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 512)            2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 512)            0         
__________

In [12]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [13]:
model.fit(X_train, Y_train, batch_size = 64, epochs=200)

Epoch 1/200
14333/14333 [==============================] - 10s 665us/step - loss: 0.4265 - acc: 0.8161
Epoch 2/200
14333/14333 [==============================] - 5s 315us/step - loss: 0.3650 - acc: 0.8515
Epoch 3/200
14333/14333 [==============================] - 4s 312us/step - loss: 0.3525 - acc: 0.8585
Epoch 4/200
14333/14333 [==============================] - 5s 319us/step - loss: 0.3420 - acc: 0.8640
Epoch 5/200
14333/14333 [==============================] - 5s 315us/step - loss: 0.3411 - acc: 0.8652
Epoch 6/200
14333/14333 [==============================] - 5s 317us/step - loss: 0.3356 - acc: 0.8664
Epoch 7/200
14333/14333 [==============================] - 4s 314us/step - loss: 0.3345 - acc: 0.8679
Epoch 8/200
14333/14333 [==============================] - 4s 313us/step - loss: 0.3326 - acc: 0.8665
Epoch 9/200
14333/14333 [==============================] - 5s 315us/step - loss: 0.3295 - acc: 0.8717
Epoch 10/200
14333/14333 [==============================] - 4s 312us/step - loss:

14333/14333 [==============================] - 4s 305us/step - loss: 0.2789 - acc: 0.8908
Epoch 160/200
14333/14333 [==============================] - 4s 306us/step - loss: 0.2751 - acc: 0.8955
Epoch 161/200
14333/14333 [==============================] - 4s 306us/step - loss: 0.2737 - acc: 0.8945
Epoch 162/200
14333/14333 [==============================] - 4s 305us/step - loss: 0.2742 - acc: 0.8936
Epoch 163/200
14333/14333 [==============================] - 4s 309us/step - loss: 0.2758 - acc: 0.8949
Epoch 164/200
14333/14333 [==============================] - 4s 307us/step - loss: 0.2721 - acc: 0.8955
Epoch 165/200
14333/14333 [==============================] - 4s 308us/step - loss: 0.2765 - acc: 0.8920
Epoch 166/200
14333/14333 [==============================] - 4s 307us/step - loss: 0.2708 - acc: 0.8944
Epoch 167/200
14333/14333 [==============================] - 4s 307us/step - loss: 0.2751 - acc: 0.8903
Epoch 168/200
14333/14333 [==============================] - 4s 306us/step - l

In [14]:
model_json = model.to_json()
with open ("LSTMModel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModel.h5")

In [15]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 1s 166us/step
Validation set accuracy =  0.8576810380663019
